In [ ]:
# pip install pandas plotly openpyxl pyyaml nbformat
import os, re, webbrowser, yaml
import pandas as pd
import plotly.graph_objects as go
import unicodedata, decimal
from collections import defaultdict

# ========= 可調參數 =========
FILE  = "bankflow.xlsm"           # 檔名，存放於此程式同目錄下
SHEET = "Raw"                     # 交易明細工作表名稱
TOP_NODE_K = 10                   # 以總流量挑出前 K 個節點上色
TOP_EDGES  = 20                   # 只取金額最大的 N 筆交易；None=不限制
MIN_AMOUNT = 10001                # 過濾金額小於此值的交易；None=不過濾

# 欄位對應（可依金融機構報表調整）
COLUMN_MAP = {
    "withdraw": ["支出金額", "轉出金額", "出金"],
    "deposit":  ["存入金額", "轉入金額", "入金"],
    "account":  ["帳號", "本行帳號"],
    "opponent": ["轉出入行庫代碼及帳號", "對方帳號", "交易對象"],
    "opponent_atm": ["ATM或末端機代號", "ATM代號"],
    "summary":  ["交易摘要", "摘要"],
    "remark":   ["備註", "附註", "說明"],
}

# ---------- (2) 備註正則化  "oppoent" & "opponent_atm" 找不到時使用----------
CP_PATTERNS = [
    # [Case 1] 'V   00400000009004021425 8123455639 ...' → 取 V 後第一個「長數字」；短數字略過
    # 避免 'V  4534703 2020557786'（第一段太短）被誤取
    (re.compile(r"^\s*[VＶ]\s+([0-9]{12,})\b"), "V_LONGSEQ"),

    # [Case 2/3] 'YSSL00400000037004727149 ...' / 'NSSL01300000038596007993 ...'
    # 取行首的「字母1~6 + 長數字」識別碼
    (re.compile(r"^\s*([A-Z]{1,6}\d{12,})\b", re.IGNORECASE), "ALNUM_ID"),

    # [Case 4] '0130VBKN   (自提卡跨行提款)' → 取括號前的字串（限定該中文說明）
    (re.compile(r"^\s*([A-Z0-9]+)\s*(?=\(自提卡跨行提款\))", re.IGNORECASE), "TOKEN_BEFORE_PAREN"),

    # [Case 5] 偵測可能的人名：剛好三個中文字，或含有全形空格、圈字符號（○、．）
    (re.compile(r"^\s*([\u4e00-\u9fff]{3}|[\u4e00-\u9fff][　○．][\u4e00-\u9fff])\s*$"), "POSSIBLE_NAME"),

    # 可自行增加其他正則化規則，在ATM or 轉出帳號均為空的情況下會自動找對手。
]

# 正則化也找不到目標，使用類別節點
CATEGORY_NODE_SCOPE = "global"    # "global" 或 "per_account"
CATEGORY_DICT_PATH  = "tx_categories.yaml"
FILL_FROM_REMARK    = True        # 結構欄位為空時，是否用 remark 正則抽對手方
ALLOW_FALLBACK_OTHER = False  # False=未命中任何類別時不落 "other"，直接丟棄

# ==========================

# ---------- 工具函式 ----------
_SCI_RE = re.compile(r'^[+-]?\d+(?:\.\d+)?[eE][+-]?\d+$')


def net_reciprocal_edges(df: pd.DataFrame) -> pd.DataFrame:
    """將互為對向的 (source,target) 與 (target,source) 合併成單一淨流向邊。"""
    if df is None or df.empty:
        return df.copy()

    g = df.groupby(["source","target"], as_index=False)["amount"].sum()
    bucket = defaultdict(lambda: {"st":0.0, "ts":0.0, "s":"", "t":""})

    for _, r in g.iterrows():
        s, t, a = r["source"], r["target"], float(r["amount"])
        key = (s, t) if s < t else (t, s)
        if not bucket[key]["s"]:
            bucket[key]["s"], bucket[key]["t"] = key[0], key[1]
        if (s, t) == (bucket[key]["s"], bucket[key]["t"]):
            bucket[key]["st"] += a
        else:
            bucket[key]["ts"] += a

    rows = []
    for key, v in bucket.items():
        a_st, a_ts = v["st"], v["ts"]
        if a_st > a_ts:
            rows.append({"source": v["s"], "target": v["t"], "amount": a_st - a_ts})
        elif a_ts > a_st:
            rows.append({"source": v["t"], "target": v["s"], "amount": a_ts - a_st})

    out = pd.DataFrame(rows)
    if not out.empty:
        out = out.sort_values("amount", ascending=False).reset_index(drop=True)
    return out



def safe_filename(s):
    return re.sub(r'[<>:"/\\|?*]+', '_', s)[:180]  # Windows 禁字 + 長度限制

def norm_text(s: str) -> str:
    t = unicodedata.normalize("NFKC", str(s or ""))
    return re.sub(r"\s+", " ", t).strip()

def clean_cp_cell(x):
    """
    專給 account/opponent/opponent_atm：
    - NFKC 半形化、去空白
    - 移除科學記號表示
    - 去除 .0 尾巴
    - 保留英數與常見分隔符（-、_）
    """
    if x is None:
        return ""
    if isinstance(x, str):
        t = unicodedata.normalize("NFKC", x)
        t = re.sub(r"\s+", "", t)
        t = re.sub(r'\.0$', '', t)
        return t

    if isinstance(x, int):
        return str(x)

    if isinstance(x, float):
        s = str(x)
        if _SCI_RE.match(s):
            s = format(decimal.Decimal(s), 'f')
        if s.endswith('.0'):
            s = s[:-2]
        return re.sub(r'[^0-9A-Za-z\-_]', '', s)

    s = str(x)
    s = unicodedata.normalize("NFKC", s)
    s = re.sub(r"\s+", "", s)
    s = re.sub(r'\.0$', '', s)
    return s


def get_col(df, aliases, required=True):
    for name in aliases:
        if name in df.columns:
            return name
    if required:
        raise KeyError(f"找不到欄位：{aliases}")
    return None

def is_zeroish(s):
    if s is None:
        return True
    s = str(s).strip().lower()
    return s in ["", "0", "0.0", "nan", "none"]

atm_pattern = re.compile(r"(?:ATM|ＡＴＭ|卡機|自動\s*櫃員\s*機|提\s*款\s*機)", re.IGNORECASE)

def read_and_clean(file, sheet, cmap=COLUMN_MAP):
    df = pd.read_excel(file, sheet_name=sheet, engine="openpyxl")

    wcol = get_col(df, cmap["withdraw"])
    dcol = get_col(df, cmap["deposit"])
    df[wcol] = pd.to_numeric(df[wcol], errors="coerce").fillna(0.0)
    df[dcol] = pd.to_numeric(df[dcol], errors="coerce").fillna(0.0)

    for key in ["account","opponent","opponent_atm","summary","remark"]:
        c = get_col(df, cmap.get(key, []), required=False)
        if not c:
            continue
        if key in ("account","opponent","opponent_atm"):
            df[c] = df[c].map(clean_cp_cell)
        else:
            df[c] = df[c].astype(str).map(norm_text)

    sum_col = get_col(df, cmap["summary"])
    is_atm = df[sum_col].str.contains(atm_pattern, regex=True, na=False)

    return df, is_atm

def extract_opponent_from_remark(remark_text: str):
    t = str(remark_text or "").strip()
    if not t:
        return None, None
    best = None
    best_score = -1
    for rx, ctype in CP_PATTERNS:
        m = rx.search(t)
        if not m:
            continue
        val = next((g for g in m.groups() if g), m.group(0)).upper()
        score = min(len(val), 12) / 12.0
        if score > best_score:
            best = (val, ctype)
            best_score = score
    return best if best else (None, None)

def load_category_rules(path=CATEGORY_DICT_PATH):
    with open(path, "r", encoding="utf-8") as f:
        y = yaml.safe_load(f)
    cats = y.get("categories", [])
    for c in cats:
        c["_patterns"]  = [re.compile(p, re.IGNORECASE) for p in c.get("patterns", [])]
        c["_negatives"] = [re.compile(n, re.IGNORECASE) for n in c.get("negatives", [])]
    cats.sort(key=lambda c: int(c.get("priority", 1000)))
    return cats

_CATS = None

def classify_single_label(text_all: str, direction: str):
    """回傳 (name, label)；若無命中且不允許 fallback，回傳 (None, None)。"""
    global _CATS
    if _CATS is None:
        _CATS = load_category_rules(CATEGORY_DICT_PATH)
    t = norm_text(text_all)
    d = str(direction or "any").upper()

    for c in _CATS:
        want = c.get("direction","any").upper()
        if want in ("W","D") and d != want:
            continue
        if any(n.search(t) for n in c["_negatives"]):
            continue
        if any(p.search(t) for p in c["_patterns"]):
            return c["name"], c.get("label", c["name"])

    if ALLOW_FALLBACK_OTHER:
        other = next((c for c in _CATS if c["name"]=="other"), None)
        if other:
            return other["name"], other.get("label","其他")
    return None, None


def category_node_name(cat_label: str, account: str, scope: str):
    if scope == "per_account":
        return f"[CAT:{account}] {cat_label}"
    return f"[CAT] {cat_label}"

def build_rowwise_with_category(df: pd.DataFrame, is_atm_series: pd.Series, cmap: dict, scope: str) -> pd.DataFrame:
    recs, decisions = [], []

    wcol = get_col(df, cmap["withdraw"])
    dcol = get_col(df, cmap["deposit"])
    acc_col = get_col(df, cmap["account"])
    opp_col = get_col(df, cmap["opponent"], required=False)
    atm_col = get_col(df, cmap["opponent_atm"], required=False)
    sum_col = get_col(df, cmap["summary"])
    rem_col = get_col(df, cmap["remark"], required=False)

    for i, row in df.iterrows():
        withdraw = float(row[wcol] or 0.0)
        deposit  = float(row[dcol] or 0.0)
        account  = str(row[acc_col]).strip()
        opponent = str(row[opp_col]).strip() if opp_col else ""
        oppatm   = str(row[atm_col]).strip() if atm_col else ""
        summary  = str(row[sum_col]).strip() if sum_col else ""
        remark   = str(row[rem_col]).strip() if rem_col else ""

        origin = "W" if (withdraw != 0 and deposit == 0) else ("D" if (deposit != 0 and withdraw == 0) else None)
        if origin is None:
            continue
        amount = withdraw if origin == "W" else deposit

        decision = {
            "row": int(i),
            "origin": origin,
            "account": account,
            "withdraw": withdraw,
            "deposit": deposit,
            "amount": amount,
            "mode": "",
            "counterparty": "",
            "summary": summary,
            "remark": remark,
            "category_name": ""
        }

        atm_flag = bool(is_atm_series.loc[i])
        cp = (oppatm if not is_zeroish(oppatm) else opponent) if atm_flag \
             else (opponent if not is_zeroish(opponent) else oppatm)

        if not is_zeroish(cp):
            decision["mode"] = "struct"
            decision["counterparty"] = cp
            s, t = (account, cp) if origin == "W" else (cp, account)
        else:
            cp_val, _cp_type = (extract_opponent_from_remark(remark) if FILL_FROM_REMARK else (None, None))
            if cp_val:
                decision["mode"] = "remark"
                decision["counterparty"] = cp_val
                s, t = (account, cp_val) if origin == "W" else (cp_val, account)
            else:
                name, label = classify_single_label((summary + " " + remark).strip(), origin)
                if not name:
                    decision["mode"] = "drop"
                    decisions.append(decision)
                    continue
                decision["mode"] = "category"
                decision["counterparty"] = label
                decision["category_name"] = name
                cat_node = category_node_name(label, account, scope)
                s, t = (account, cat_node) if origin == "W" else (cat_node, account)

        recs.append({"source": s, "target": t, "amount": float(amount), "origin": origin})
        decisions.append(decision)

    out = pd.DataFrame(recs)

    audit = pd.DataFrame(
        decisions,
        columns=["row","origin","account","withdraw","deposit","amount","mode","counterparty","summary","remark","category_name"]
    )
    audit.to_csv("sankey_decisions_audit.csv", index=False, encoding="utf-8-sig")

    if out.empty:
        return out
    out["source"] = out["source"].astype(str).str.replace(r"\s+", " ", regex=True)
    out["target"] = out["target"].astype(str).str.replace(r"\s+", " ", regex=True)
    return out

# ---------- 匯總與圖 ----------
def prefer_withdraw(edges_rowwise: pd.DataFrame) -> pd.DataFrame:
    if edges_rowwise.empty:
        return edges_rowwise

    w = (edges_rowwise[edges_rowwise["origin"]=="W"]
         .groupby(["source","target"], as_index=False)["amount"].sum()
         .rename(columns={"amount":"amount_W"}))
    d = (edges_rowwise[edges_rowwise["origin"]=="D"]
         .groupby(["source","target"], as_index=False)["amount"].sum()
         .rename(columns={"amount":"amount_D"}))
    g = pd.merge(w, d, on=["source","target"], how="outer").fillna(0.0)
    g["amount"] = g["amount_W"].where(g["amount_W"] > 0, g["amount_D"])
    edges = g[["source","target","amount"]].copy()
    edges = edges[edges["amount"] > 0]
    edges = edges.sort_values("amount", ascending=False).reset_index(drop=True)
    return edges

def compute_top_nodes(edges, k=10):
    outflow = edges.groupby("source")["amount"].sum().rename("out")
    inflow  = edges.groupby("target")["amount"].sum().rename("in")
    strength = pd.concat([outflow, inflow], axis=1).fillna(0.0)
    strength["total"] = strength["out"] + strength["in"]
    strength = strength.sort_values("total", ascending=False)
    top_nodes = list(strength.head(k).index)
    return top_nodes, strength

def build_color_map(nodes, top_nodes):
    palette = [
        "rgba(31,119,180,1.0)","rgba(255,127,14,1.0)","rgba(44,160,44,1.0)",
        "rgba(214,39,40,1.0)","rgba(148,103,189,1.0)","rgba(140,86,75,1.0)",
        "rgba(227,119,194,1.0)","rgba(127,127,127,1.0)","rgba(188,189,34,1.0)",
        "rgba(23,190,207,1.0)",
    ]
    base_node = "rgba(180,180,180,0.65)"
    cmap = {n: base_node for n in nodes}
    for i, n in enumerate(top_nodes or []):
        if n in cmap:
            cmap[n] = palette[i % len(palette)]
    return cmap

def _rgba_set_alpha(rgba: str, alpha: float) -> str:
    m = re.match(r"rgba\(\s*(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*,\s*([\d.]+)\s*\)", rgba)
    if not m:
        return rgba
    r, g, b = m.group(1), m.group(2), m.group(3)
    return f"rgba({r},{g},{b},{alpha})"

def choose_link_colors(sub_edges, node_colors):
    base_node = "rgba(180,180,180,0.65)"
    base_link = "rgba(160,160,160,0.25)"
    colors = []
    for s, t in zip(sub_edges["source"], sub_edges["target"]):
        cs = node_colors.get(s, base_node)
        ct = node_colors.get(t, base_node)
        if cs != base_node and ct == base_node:
            colors.append(_rgba_set_alpha(cs, 0.40))
        elif ct != base_node and cs == base_node:
            colors.append(_rgba_set_alpha(ct, 0.40))
        elif cs != base_node and ct != base_node:
            colors.append(_rgba_set_alpha(cs, 0.45))
        else:
            colors.append(base_link)
    return colors

def compute_inout_strength(edges: pd.DataFrame):
    outflow = edges.groupby("source")["amount"].sum().rename("out")
    inflow  = edges.groupby("target")["amount"].sum().rename("in")
    s = pd.concat([outflow, inflow], axis=1).fillna(0.0)
    s["net"] = s["out"] - s["in"]     # >0 淨流出；<0 淨流入
    return s

def compute_node_positions_lr(nodes: pd.Index, edges: pd.DataFrame):
    """
    左欄 = 淨流出(net>0)；右欄 = 淨流入(net<0)；中欄 = 近似平衡(|net| 很小)。
    每欄依字母序等距分佈 y。
    """
    s = compute_inout_strength(edges)  
    left, mid, right = [], [], []
    eps = max(1.0, 0.001 * (s["out"].sum() + s["in"].sum()))  

    for n in nodes:
        net = float(s.loc[n, "net"]) if n in s.index else 0.0
        if net >  eps:   
            left.append(n)
        elif net < -eps:  
            right.append(n)
        else:
            mid.append(n)

    def place(col_nodes, x):
        m = max(len(col_nodes), 1)
        y_map = {n: (i + 1) / (m + 1) for i, n in enumerate(sorted(col_nodes, key=str))}
        return {n: x for n in col_nodes}, y_map

    xL, yL = place(left,  0.10)
    xM, yM = place(mid,   0.50)
    xR, yR = place(right, 0.90)
    x_map = {**xL, **xM, **xR}
    y_map = {**yL, **yM, **yR}
    xs = [x_map.get(n, 0.50) for n in nodes]
    ys = [y_map.get(n, 0.50) for n in nodes]
    sizes = (len(left), len(mid), len(right))
    return xs, ys, sizes

def _as_list(x):
    if x is None: return []
    if isinstance(x, (list, tuple, set)): return [str(i) for i in x]
    return [str(x)]

def subgraph_by_accounts(
    edges: pd.DataFrame,
    seeds,                      
    hops: int = 1,             
    direction: str = "both",   
    include_categories: bool = True, 
):
    if edges is None or edges.empty:
        return edges.copy()

    seeds = set(_as_list(seeds))
    keep_mask = pd.Series(False, index=edges.index)
    frontier = set(seeds)

    for _ in range(max(1, int(hops))):
        if not frontier: break
        if direction == "out":
            hit = edges["source"].isin(frontier)
        elif direction == "in":
            hit = edges["target"].isin(frontier)
        else:  
            hit = edges["source"].isin(frontier) | edges["target"].isin(frontier)

        keep_mask |= hit

        next_frontier = set()
        if direction in ("out", "both"):
            next_frontier |= set(edges.loc[hit, "target"])
        if direction in ("in", "both"):
            next_frontier |= set(edges.loc[hit, "source"])
        frontier = next_frontier

    sub = edges.loc[keep_mask].copy()

    if not include_categories:
        mask = ~sub["source"].astype(str).str.startswith("[CAT") & ~sub["target"].astype(str).str.startswith("[CAT")
        sub = sub.loc[mask].copy()

    if not sub.empty:
        sub = sub.sort_values("amount", ascending=False).reset_index(drop=True)
    return sub

def view_accounts(
    edges: pd.DataFrame,
    accounts,              
    hops: int = 1,
    direction: str = "both",
    include_categories: bool = True,
    min_amount: float | None = None,
    top_edges: int | None = None,
    per_account: bool = False, 
    title_prefix: str = "Account",
    net_pairs: bool = False, 
):
    accounts_list = _as_list(accounts)

    def _prep_and_plot(sub_edges, title):
        _e = sub_edges.copy()
        if min_amount is not None:
            _e = _e[_e["amount"] >= float(min_amount)]
        if net_pairs:                   
            _e = net_reciprocal_edges(_e)
        if top_edges is not None:
            _e = _e.nlargest(int(top_edges), "amount")
        tn, _ = compute_top_nodes(_e, k=min(TOP_NODE_K, max(len(_e), 1)))
        plot_sankey(_e, title=title, out_html=None, top_nodes=tn)

    if per_account:
        for acc in accounts_list:
            sg = subgraph_by_accounts(edges, acc, hops=hops, direction=direction, include_categories=include_categories)
            _prep_and_plot(sg, f"{title_prefix} = {acc} | hops={hops} | dir={direction} | cats={'on' if include_categories else 'off'}")
    else:
        sg_all = []
        for acc in accounts_list:
            sg = subgraph_by_accounts(edges, acc, hops=hops, direction=direction, include_categories=include_categories)
            if not sg.empty:
                sg_all.append(sg)

        if sg_all:
            sg_all = pd.concat(sg_all, ignore_index=True)
        else:
            sg_all = pd.DataFrame(columns=edges.columns)

        if not sg_all.empty:
            sg_all = (sg_all.sort_values("amount", ascending=False)
                            .drop_duplicates(["source","target"], keep="first")
                            .reset_index(drop=True))
        title = f"{title_prefix}s = {', '.join(accounts_list)} | hops={hops} | dir={direction} | cats={'on' if include_categories else 'off'}"
        safe_title = safe_filename(title)
        _prep_and_plot(sg_all, safe_title)



def plot_sankey(sub_edges, title, out_html=None, top_nodes=None, width=None, height=None):
    if sub_edges is None or sub_edges.empty:
        print(f"{title}: no edges"); return

    nodes = pd.Index(pd.unique(sub_edges[["source","target"]].values.ravel()))
    node_ix = {n:i for i,n in enumerate(nodes)}
    node_colors = build_color_map(nodes, set(top_nodes or []))
    link_colors = choose_link_colors(sub_edges, node_colors)

    x_positions, y_positions, col_sizes = compute_node_positions_lr(nodes, sub_edges)

    # 初始高度：依最大欄位節點數估，避免擠；可滾動
    if height is None:
        max_col = max(col_sizes) if col_sizes else 1
        height = max(1200, 70 * max_col + 240)  

    link_src = sub_edges["source"].map(node_ix).tolist()
    link_tar = sub_edges["target"].map(node_ix).tolist()
    link_val = sub_edges["amount"].tolist()
    node_labels = [str(n) for n in nodes]
    node_color_list = [node_colors[n] for n in nodes]

    fig = go.Figure(data=[go.Sankey(
        orientation="h",
        arrangement="freeform",  
        node=dict(
            pad=18, thickness=20, line=dict(width=0.5, color="rgba(50,50,50,0.6)"),
            x=x_positions, y=y_positions,
            label=node_labels, color=node_color_list,
            hovertemplate="%{label}<extra></extra>",
        ),
        link=dict(
            source=link_src, target=link_tar, value=link_val, color=link_colors,
            hovertemplate="From %{source.label} → %{target.label}<br>Amount: %{value:,.0f}<extra></extra>",
        )
    )])

    fig.update_layout(
        title_text=title, font_size=12,
        width=1200 if width is None else width,  
        height=height,
        margin=dict(l=10, r=10, t=60, b=10),
    )

    if not out_html:
        out_html = f"{title.replace(' ','_')}.html"
    abs_path = os.path.abspath(out_html)

    style_css = """
    <style>
    html, body { height:auto !important; overflow-y:auto !important; }
    #sankey_graph { display:block; }
    </style>
    """
    custom_js = r"""
    <script>
    document.addEventListener('DOMContentLoaded', function() {
    const gd = document.getElementById('sankey_graph');

    function toast(msg) {
        const tip = document.createElement('div');
        tip.textContent = msg;
        tip.style.position = 'fixed';
        tip.style.bottom = '20px';
        tip.style.left = '50%';
        tip.style.transform = 'translateX(-50%)';
        tip.style.background = 'rgba(0,0,0,0.75)';
        tip.style.color = 'white';
        tip.style.padding = '6px 12px';
        tip.style.borderRadius = '6px';
        tip.style.zIndex = '9999';
        tip.style.fontSize = '13px';
        tip.style.pointerEvents = 'none';
        document.body.appendChild(tip);
        setTimeout(() => tip.remove(), 1200);
    }

    function copyText(txt) {
        if (!txt) return;
        if (navigator.clipboard && window.isSecureContext) {
        navigator.clipboard.writeText(txt).then(() => toast('已複製：' + txt))
            .catch(() => fallback());
        } else {
        fallback();
        }
        function fallback() {
        const ta = document.createElement('textarea');
        ta.value = txt;
        ta.style.position = 'fixed';
        ta.style.top = '-1000px';
        document.body.appendChild(ta);
        ta.focus(); ta.select();
        try { document.execCommand('copy'); toast('已複製：' + txt); }
        catch(e) { console.warn('copy failed', e); }
        finally { ta.remove(); }
        }
    }

    function fitWidth() {
        const w = Math.max(480, Math.floor(window.innerWidth * 0.98));
        Plotly.relayout(gd, {width: w});
    }

    function fmtAmount(v) {
        try { return Number(v).toLocaleString(); } catch(e) { return String(v); }
    }

    function placeTag(id, text, xNorm, yNorm, side) {
        const L = gd._fullLayout.margin.l || 0;
        const R = gd._fullLayout.margin.r || 0;
        const T = gd._fullLayout.margin.t || 0;
        const B = gd._fullLayout.margin.b || 0;
        const W = gd._fullLayout.width  || gd.clientWidth  || window.innerWidth;
        const H = gd._fullLayout.height || gd.clientHeight || window.innerHeight;
        const plotW = Math.max(0, W - (L + R));
        const plotH = Math.max(0, H - (T + B));

        const xpx = L + xNorm * plotW + (side === 'left' ? -8 : +8);
        const ypx = T + yNorm * plotH;

        let tag = document.getElementById(id);
        if (!tag) {
        tag = document.createElement('div');
        tag.id = id;
        tag.style.position = 'fixed';
        tag.style.transform = 'translateY(-50%)';
        tag.style.background = 'rgba(0,0,0,0.80)';
        tag.style.color = 'white';
        tag.style.padding = '2px 6px';
        tag.style.borderRadius = '6px';
        tag.style.fontSize = '12px';
        tag.style.lineHeight = '1.2';
        tag.style.whiteSpace = 'nowrap';
        tag.style.pointerEvents = 'none';
        tag.style.zIndex = '9998';
        document.body.appendChild(tag);
        }
        tag.textContent = text;
        tag.style.left = (xpx + (side === 'left' ? -tag.offsetWidth : 0)) + 'px';
        tag.style.top  = ypx + 'px';
        return tag;
    }

    function clearTags() {
        ['sankey_src_tag','sankey_dst_tag'].forEach(id => {
        const el = document.getElementById(id);
        if (el) el.remove();
        });
    }

    gd.on('plotly_click', function(ev) {
        if (!ev || !ev.points || !ev.points.length) return;
        const p = ev.points[0];
        const labels = ((gd._fullData || [])[0] || {}).node?.label || [];
        if (p && p.data && p.data.type === 'sankey') {
        if (p.label) {
            copyText(String(p.label).trim());
        } else if (Number.isInteger(p.source) && Number.isInteger(p.target)) {
            const s = labels[p.source] || ('#' + p.source);
            const t = labels[p.target] || ('#' + p.target);
            copyText(String(s).trim() + ' → ' + String(t).trim());
        }
        }
    });

    gd.on('plotly_hover', function(ev) {
        if (!ev || !ev.points || !ev.points.length) return;
        const p = ev.points[0];
        if (!p || !p.data || p.data.type !== 'sankey') return;
        if (!Number.isInteger(p.source) || !Number.isInteger(p.target)) return;

        const d0 = (gd._fullData || [])[0] || {};
        const labels = d0.node?.label || [];
        const xs = d0.node?.x || [];
        const ys = d0.node?.y || [];

        const sIdx = p.source, tIdx = p.target;
        const sLabel = (labels[sIdx] || '').trim();
        const tLabel = (labels[tIdx] || '').trim();
        const amount = fmtAmount(p.value);

        placeTag('sankey_src_tag', `${sLabel} → ${tLabel}  ${amount}`, xs[sIdx], ys[sIdx], 'right');
        placeTag('sankey_dst_tag', `${sLabel} → ${tLabel}  ${amount}`, xs[tIdx], ys[tIdx], 'left');
    });

    gd.on('plotly_unhover', function() { clearTags(); });
    window.addEventListener('resize', function() { clearTags(); });

    setTimeout(fitWidth, 50);
    window.addEventListener('resize', fitWidth);
    });
    </script>
    """

    html_content = fig.to_html(
        include_plotlyjs="inline",
        full_html=True,
        div_id="sankey_graph",
        config={
            "displaylogo": False,
            "modeBarButtonsToRemove": ["lasso2d","select2d"],
            "scrollZoom": True,
            "responsive": True
        }
    )
    html_content = html_content.replace("</head>", style_css + "</head>")
    html_content = html_content.replace("</body>", "<div style='height:200px'></div>" + custom_js + "</body>")

    with open(out_html, "w", encoding="utf-8") as f:
        f.write(html_content)
    print(f"[ok] wrote: {out_html}")
    webbrowser.open('file://' + abs_path)

# ---------- 主流程 ----------
if __name__ == "__main__":
    df, is_atm = read_and_clean(FILE, SHEET)

    rowwise = build_rowwise_with_category(df, is_atm, cmap=COLUMN_MAP, scope=CATEGORY_NODE_SCOPE)

    edges = prefer_withdraw(rowwise)

    _edges = edges.copy()
    if MIN_AMOUNT is not None:
        _edges = _edges[_edges["amount"] >= float(MIN_AMOUNT)]
    if TOP_EDGES is not None:
        _edges = _edges.nlargest(int(TOP_EDGES), "amount")

    top_nodes, node_strength = compute_top_nodes(_edges, k=TOP_NODE_K)



    # 全帳戶圖（用註解方式開關，這裡打開下面 view_accounts 就要註解掉，註解快捷鍵：ctrl+/）
    # plot_sankey(
    #     sub_edges=_edges,
    #     title=f"All Flows (A-mode, node_scope={CATEGORY_NODE_SCOPE})",
    #     out_html=None,
    #     top_nodes=top_nodes
    # )

    # 單一或多帳戶視角分析：
    view_accounts(
        edges=edges,
        accounts=["20181000619174", "28881006563125", "0222979189099"],   # 單帳號或多帳號均可
        hops=1,                        # 可選擇看幾層帳戶
        direction="both",              # 下面設定可以不用理他，有興趣再去問 GPT。
        include_categories=True,
        min_amount=MIN_AMOUNT,            
        top_edges=TOP_EDGES,           
        per_account=False,
        net_pairs=False                
    )




[ok] wrote: Accounts_=_20181000619174,_28881006563125,_0222979189099___hops=1___dir=both___cats=on.html


In [3]:
import plotly.graph_objects as go, plotly.io as pio
pio.renderers.default = "browser"
fig = go.Figure(data=[go.Sankey(node=dict(label=["A","B"]),
                                link=dict(source=[0], target=[1], value=[10]))])
fig.write_html("mini.html", include_plotlyjs="inline", full_html=True)
webbrowser.open('file://' + os.path.abspath("mini.html"))



True